# Computing the unary epsilon indicator of two Pareto frontier
This program computes the (additive) unary epsilon indicator of a Pareto-optimal frontier, which measures the distance a frontier must be translated in order to "cover" (at least weakly dominate) the frontier's ideal solution. Generally, this indicator measures the distance to a Pareto set, but as we begin with a Pareto set, that measure does not apply. Since we use this to compare across frontiers with varying bounds on their objective spaces, this program first normalizes the objective space that it is entirely within the [0,1]^N hypercube (N objectives).

In [1]:
import pandas as pd
import numpy as np
import time

### Functions used

In [2]:
def computeAdditiveUnaryEpsilonIndicator(f1):
    '''
    Computes the unary epsilon indicator for a frontier
    The unary epsilon indicator is the addend by which each component of each solution in f1 must be increased
    such that the ideal solution is covered
    '''
        
    # get list of objectives (assumed to be column headers)
    # objectives are assumed to be max
    objectives = f1.columns.tolist()
    f2 = pd.DataFrame(columns=objectives)
    f2.loc[0] = np.ones(len(objectives))
    
    # initialize the result
    epsilon = -np.Inf
    for idx2,row2 in f2.iterrows(): # for each solution z2 in f2...
        # what's the minimimum amount that f1 would have to be translated
        # so that at least one solution z1 in f1 weakly dominates z2?
        minTranslationToCover = np.Inf
        for idx1,row1 in f1.iterrows():
            minTranslationToCover = min(minTranslationToCover,max(row2[objectives] - row1[objectives]))
            
        epsilon = max(epsilon,minTranslationToCover)                
    
    return epsilon

In [3]:
def maxAllAndNorm(origf,objSenses):
    
    f = origf
    
    # get bounds on the objectives' values
    objBounds = {}
    for obj,sense in objSenses.items():
        objBounds[obj] = [f[obj].min(),f[obj].max()]
    # get each objective's ideal value
    idealObjVals = {}
    for obj,bounds in objBounds.items():
        idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max
    # get each objective's worst value
    worstObjVals = {}
    for obj,bounds in objBounds.items():
        worstObjVals[obj] = bounds[not(objSenses[obj])]
        
    # Normalize the objective spaces by converting each value to the relative achievement along its axis:

    #   distance from the worst case value
    # ---------------------------------------
    # total distance spanned by the objective

    # Normalized objective space is in [0,1]^N where N = # of objectives


    for obj,bounds in objBounds.items():
        f[obj] = f[obj].apply(
            lambda x: \
            abs(x - worstObjVals[obj]) / \
            (bounds[1]-bounds[0]))
    
    return f

### Preparation for I_eps1 computation algorithm

In [4]:
# Get the solutions that define the pareto frontiers.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

f1 = pd.read_csv("../solutionSets/3d/ClimateChange_None/OptimalSolutions_NONE_nondominated.txt")

In [5]:
# Hard-coded manipulations to tidy-up sols

# set index, remove unwanted columns
f1 = f1.drop(["SolutionIndex"],axis=1)

In [6]:
# Get list of objectives from the dataframe (assumed to be identical across the frontiers)

objs = f1.columns.tolist()

In [7]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the dataframe

objSenses = {}

# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0
objSenses[" MinOwlHabitat"] = 1
#objSenses["PER"] = 0
#objSenses

### Running the algorithm

In [8]:
# Compute the binary epsilon indicator

f1 = maxAllAndNorm(f1,objSenses)

computeAdditiveUnaryEpsilonIndicator(f1)

0.20485718615650172